In [7]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score, ConfusionMatrixDisplay

In [46]:
data = pd.read_csv('50_Startups.csv')
data.dtype

R&D Spend          float64
Administration     float64
Marketing Spend    float64
State               object
Profit             float64
dtype: object

### Soal Tipe Data

In [83]:
# Data

company_name_list = [{'name': 'Company 1'},
          {'name': 'Company 2'},
          {'name': 'Company 3'}]

employee_name_list = [{'name': 'John Doe'},
          {'name': 'Tom Smith'},
          {'name': 'Andrew Sebastian'}]

company_detail_list = {
      'Company 1': {
          'name': 'Company 1',
          'domain': 'Retail',
          'clients': [
              {
                  'name': 'acme.inc',
                  'country': 'united states'
              },
              {
                  'name': 'Wayne.co',
                  'country': 'united states'
              }
          ]
      },
      'Company 2': {
          'name': 'Company 2',
          'domain': 'Construction',
          'clients': [
              {
                  'name': 'Tesla',
                  'country': 'united states'
              },
              {
                  'name': 'Japan Airlines',
                  'country': 'japan'
              },
              {
                  'name': 'Indofood',
                  'country': 'indonesia'
              }
          ]
      },
      'Company 3': {
          'name': 'Company 3',
          'domain': 'Healthcare',
          'clients': [
              {
                  'name': 'Petronas',
                  'country': 'malaysia'
              },
              {
                  'name': 'VW Group',
                  'country': 'germany'
              },
              {
                  'name': 'IBM',
                  'country': 'united states'
              },
              {
                  'name': 'Mitsubishi',
                  'country': 'japan'
              }
          ]
      }
  }

employee_detail_list = {
      'John Doe': {
          'name': 'EMP-0001',
          'first_name': 'John',
          'last_name': 'Doe',
          'full_name': 'John Doe',
          'company': 'Company 1'
      },
      'Tom Smith': {
          'name': 'EMP-0002',
          'first_name': 'Tom',
          'last_name': 'Smith',
          'full_name': 'Tom Smith',
          'company': 'Company 2'
      },
      'Andrew Sebastian': {
          'name': 'EMP-0003',
          'first_name': 'Andrew',
          'last_name': 'Sebastian',
          'full_name': 'Andrew Sebastian',
          'company': 'Company 2'
      },
  }

### 1

In [84]:
sorted_companies = sorted(
    [{'name': company['name'], 'domain': company_detail_list[company['name']]['domain']}
     for company in company_name_list],
    key=lambda x: x['domain'],
    reverse=True
)

print(sorted_companies)


[{'name': 'Company 1', 'domain': 'Retail'}, {'name': 'Company 3', 'domain': 'Healthcare'}, {'name': 'Company 2', 'domain': 'Construction'}]


### 2

In [85]:
for company in company_name_list:
    company_name = company['name']
    domain = company_detail_list[company_name]['domain']
    num_clients = len(company_detail_list[company_name]['clients'])
    print(f"{company_name}: {domain}, relation: {num_clients} clients")


Company 1: Retail, relation: 2 clients
Company 2: Construction, relation: 3 clients
Company 3: Healthcare, relation: 4 clients


### 3

In [86]:
def get_employee_company_domain():
    return [{'full_name': employee_detail['full_name'],
             'company': employee_detail['company'],
             'domain': company_detail_list[employee_detail['company']]['domain']}
            for employee_name, employee_detail in employee_detail_list.items()]

print(get_employee_company_domain())


[{'full_name': 'John Doe', 'company': 'Company 1', 'domain': 'Retail'}, {'full_name': 'Tom Smith', 'company': 'Company 2', 'domain': 'Construction'}, {'full_name': 'Andrew Sebastian', 'company': 'Company 2', 'domain': 'Construction'}]


### 4

In [87]:
def get_companies_with_employees():
    return [{'company': company['name'],
             'employees': [employee_detail['full_name'] for employee_detail in employee_detail_list.values() if employee_detail['company'] == company['name']]}
            for company in company_name_list]

print(get_companies_with_employees())


[{'company': 'Company 1', 'employees': ['John Doe']}, {'company': 'Company 2', 'employees': ['Tom Smith', 'Andrew Sebastian']}, {'company': 'Company 3', 'employees': []}]


### Soal pre-processing data

In [90]:
data_startup = pd.read_csv('50_Startups.csv')
data_startup.sample(7)

,R&D Spend,Administration,Marketing Spend,State,Profit
27,72107.60,127864.55,353183.81,New York,105008.31
41,27892.92,84710.77,164470.71,Florida,77798.83
39,38558.51,82982.09,174999.30,California,81005.76
12,93863.75,127320.38,249839.44,Florida,141585.52
31,NaN,152701.92,88218.23,New York,NaN
17,94657.16,145077.58,282574.31,New York,125370.37
21,78389.47,153773.43,299737.29,New York,111313.02


### 1

In [110]:
# Membaca dataset
df = pd.read_csv("50_Startups.csv")

# Menampilkan field mana saja yang memiliki data kosong
fields_with_missing_data = df.columns[df.isnull().any()]
print("Fields dengan data kosong:", fields_with_missing_data)

# Mengisi data kosong dengan nilai mean
for field in fields_with_missing_data:
    mean_value = df[field].mean()
    df[field].fillna(mean_value, inplace=True)

# Menyimpan hasil perubahan
df.to_csv("50_Startups.csv", index=False)

df.sample(5)


Fields dengan data kosong: Index([], dtype='object')


,R&D Spend,Administration,Marketing Spend,Profit,State
3,1.584537e+00,-0.161817,1.478634e+00,1.788670,New York
40,-1.126199e+00,-0.166760,-4.704726e-01,-0.846135,California
10,-1.444366e-16,-0.479207,5.167498e-02,0.862754,Florida
2,1.797168e+00,-0.850460,1.707769e+00,1.993801,Florida
19,2.258009e-01,1.207206,1.598721e-16,0.275056,New York


### 2

In [109]:
# Membaca dataset
df = pd.read_csv("50_Startups.csv")

# Melakukan OneHotEncoder ke field State
df = pd.get_dummies(df, columns=['State'])

# Menyimpan hasil perubahan
df.to_csv("50_Startups_onehot.csv", index=False)

df.sample(5)

,R&D Spend,Administration,Marketing Spend,Profit,State_California,State_Florida,State_New York
33,-0.499272,-7.753361e-01,-8.288993e-02,-0.379427,False,True,False
7,1.254557,6.655787e-16,9.290873e-01,1.105200,False,True,False
37,-0.767115,6.655787e-16,-2.459799e-01,-0.551363,True,False,False
21,0.037527,1.217395e+00,7.054687e-01,-0.013540,False,False,True
48,-1.787653,-2.791579e+00,1.598721e-16,-1.917721,False,False,True


### 3

In [107]:
# Membaca dataset
df = pd.read_csv("50_Startups.csv")

# Menghitung nilai Tax
df['Tax'] = (df['Profit'] + df['Marketing Spend'] + df['Administration']) * 0.05

# Menyimpan hasil perubahan
df.to_csv("50_Startups_tax.csv", index=False)

df.sample(5)

,R&D Spend,Administration,Marketing Spend,Profit,State,Tax
8,1.025831e+00,1.018794e+00,8.154837e-01,1.016062,New York,0.142517
10,-1.700419e-16,-4.792072e-01,5.167498e-02,0.862754,Florida,0.021761
23,-1.700419e-16,-6.695015e-01,1.865175e-16,-0.078465,Florida,-0.037398
20,-1.254345e-02,-3.505970e-01,6.955305e-01,0.166734,California,0.025583
24,5.982100e-03,6.727952e-16,-7.689550e-01,-0.083046,New York,-0.042600


### 4

In [108]:
from sklearn.preprocessing import StandardScaler

# Membaca dataset
df = pd.read_csv("50_Startups.csv")

# Menghapus kolom non-numerik (State)
df_numerical = df.drop(columns=['State'])

# Inisialisasi StandardScaler
scaler = StandardScaler()

# Melakukan scaling ke field-field numerik
scaled_data = scaler.fit_transform(df_numerical)

# Membuat DataFrame baru dari data yang sudah di-scale
df_scaled = pd.DataFrame(scaled_data, columns=df_numerical.columns)

# Menambahkan kolom State yang telah dihapus sebelumnya
df_scaled['State'] = df['State']

# Menyimpan hasil perubahan
df_scaled.to_csv("50_Startups.csv", index=False)

df.sample(5)


,R&D Spend,Administration,Marketing Spend,Profit,State
23,-1.700419e-16,-0.669502,1.865175e-16,-0.078465,Florida
26,-3.423040e-02,0.838720,-8.293979e-01,-0.154000,Florida
8,1.025831e+00,1.018794,8.154837e-01,1.016062,New York
9,1.091300e+00,-0.554449,7.540503e-01,0.954339,California
35,-7.215349e-01,-1.482987,-1.673481e-01,-0.386964,New York
